In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

- Yolov5 torch_버전 불러오기

In [1]:
# Yolov5 requirements
!pip install -r https://raw.githubusercontent.com/ultralytics/yolov5/master/requirements.txt

/bin/pip:6: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import load_entry_point


In [3]:
# torch_ver Yolov5
import torch

yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s',
                            device='cuda:0' if torch.cuda.is_available() else 'cpu')  # 예측 모델
yolo_model.classes = [0]  # 예측 클래스 (0 : 사람)

Using cache found in /home/nkw/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-11 Python-3.8.10 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24257MiB)

100%|██████████| 14.1M/14.1M [00:01<00:00, 11.7MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [ ]:
# colab 환경에서는 mediapipe install
# !pip install mediapipe

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os
import random
import gc
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

from IPython.display import Image, clear_output
from torch.cuda import memory_allocated, empty_cache
from torch.optim import Adam
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from glob import glob
from tqdm import tqdm
# from google.colab.patches import cv2_imshow

# %matplotlib inline

# 이미지 저장 및 출력 함수
def save_and_display_image(img, file_name='output_image.png'):
    # 이미지를 파일로 저장
    cv2.imwrite(file_name, img)
    # 저장된 이미지를 matplotlib을 통해 출력
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # OpenCV는 BGR 포맷을 사용하므로, RGB로 변환 필요
    plt.imshow(img)
    plt.axis('off')  # 축 제거
    plt.show()



- 데이터 전처리 및 세부 조정

In [2]:
BATCH_SIZE = 6
EPOCH = 700
NUM_LAYERS = 1      # LSTM model: num_layers
start_dot = 11      # mp.solutions.pose 시작 포인트 (0: 얼굴부터 발목까지, 11: 어깨부터 발목까지)
n_CONFIDENCE = 0.3    # MediaPipe Min Detectin confidence check
y_CONFIDENCE = 0.3    # Yolv5 Min Detectin confidence check

mp_pose = mp.solutions.pose
attention_dot = [n for n in range(start_dot, 29)]

# 라인 그리기
if start_dot == 11:
    """몸 부분만"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24]]
    print('Pose : Only Body')

else:
    """얼굴 포함"""
    draw_line = [[11, 13], [13, 15], [15, 21], [15, 19], [15, 17], [17, 19], \
                [12, 14], [14, 16], [16, 22], [16, 20], [16, 18], [18, 20], \
                [23, 25], [25, 27], [24, 26], [26, 28], [11, 12], [11, 23], \
                [23, 24], [12, 24], [9, 10], [0, 5], [0, 2], [5, 8], [2, 7]]
    print('Pose : Face + Body')

Pose : Only Body


In [13]:
# Yolov4 바운딩 box 안에서 media pipe 데이터 전처리 함수

def get_skeleton(video_path, attention_dot, draw_line):
    frame_length = 30 # LSTM 모델에 넣을 frame 수

    xy_list_list, xy_list_list_flip = [], []
    cv2.destroyAllWindows()
    pose = mp_pose.Pose(static_image_mode = True, model_complexity = 1, \
                        enable_segmentation = False, min_detection_confidence = n_CONFIDENCE)
    cap = cv2.VideoCapture(video_path)

    if cap.isOpened():

        while True:
            ret, img = cap.read()

            if ret == True:

                """ Yolo 바운딩 박스 및 좌표 추출"""
                img = cv2.resize(img, (640, 640))
                res = yolo_model(img)
                res_refine = res.pandas().xyxy[0].values
                nms_human = len(res_refine)
                if nms_human > 0:
                    for bbox in res_refine:
                        """바운딩 박스 상하좌우 크기 조절"""
                        xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
                        if xx1 < 0:
                            xx1 = 0
                        elif xx2 > 639:
                            xx2 = 639
                        if yy1 < 0:
                            yy1 = 0
                        elif yy2 > 639:
                            yy2 = 639

                        start_point = (xx1, yy1)
                        end_point = (xx2, yy2)

                        """ Yolov5 바운딩 박스 좌표 안에서 mediapipe Pose 추출"""
                        if bbox[4] > y_CONFIDENCE: # bbox[4] : confidence 데이터
                            # img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2) # 바운딩 박스 그리기 : 데이터 추출 확인용
                            c_img = img[yy1:yy2, xx1:xx2] # 바운딩 박스 좌표
                            results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolov5 바운딩 박스 좌표 안에서 'mp_pose' 좌표

                            if not results.pose_landmarks: continue
                            idx = 0
                            draw_line_dic = {}
                            xy_list, xy_list_flip = [], []
                            # 33 반복문 진행 : 33개 중 18개의 dot
                            for x_and_y in results.pose_landmarks.landmark:
                                if idx in attention_dot:
                                    xy_list.append(x_and_y.x)
                                    xy_list.append(x_and_y.y)
                                    xy_list_flip.append(1 - x_and_y.x)
                                    xy_list_flip.append(x_and_y.y)

                                    x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                                    draw_line_dic[idx] = [x, y]
                                idx += 1

                            if len(xy_list) != len(attention_dot) * 2:
                                print('Error : attention_dot 데이터 오류')

                            xy_list_list.append(xy_list)
                            xy_list_list_flip.append(xy_list_flip)

                            """mediapipe line 그리기 부분 : 데이터 추출(dot) 확인용"""
                            # for line in draw_line:
                            #     x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                            #     x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                            #     c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 4)
                            # # cv2.imshow('Landmark Image', img)
                            # cv2_imshow(img)
                            # cv2.waitKey(1)

            elif ret == False: break


        # 부족한 프레임 수 맞추기
        if len(xy_list_list_flip) < 15:
            return False, False
        elif len(xy_list_list_flip) < frame_length:
            f_ln = frame_length - len(xy_list_list_flip)
            for _ in range(f_ln):
                xy_list_list.append(xy_list_list[-1])
                xy_list_list_flip.append(xy_list_list_flip[-1])

    cap.release()
    cv2.destroyAllWindows()


    return xy_list_list, xy_list_list_flip

In [4]:
# video_path = '/content/drive/MyDrive/Colab_Notebooks/LSTM/datasets' # dataset 경로
video_path = '/home/nkw/ihs/ihs2/zerobase_DL_project/datas/dataset' # dataset 경로
video_path

'/home/nkw/ihs/ihs2/zerobase_DL_project/datas/dataset'

In [11]:
# 폴더 내용물 출력
for fold in os.listdir(video_path):
    print(f"Directory: {fold}")
    for video_name in os.listdir(os.path.join(video_path, fold)):
        print(f"  File: {video_name}")

Directory: abnormal
  File: 08_bag_30.mp4
  File: 03_pocket_30.mp4
  File: 09_bag_30.mp4
  File: 04_pocket_30.mp4
  File: 02_pocket_30.mp4
  File: 01_pocket_30.mp4
  File: 07_bag_30.mp4
  File: 05_pocket_30.mp4
  File: 06_pocket_30.mp4
  File: 10_bag_30.mp4
Directory: normal
  File: 06_normal_30.mp4
  File: 05_normal_30.mp4
  File: 01_normal_30.mp4
  File: 08_normal_30.mp4
  File: 02_normal_30.mp4
  File: 10_normal_30.mp4
  File: 04_normal_30.mp4
  File: 09_normal_30.mp4
  File: 07_normal_30.mp4
  File: 03_normal_30.mp4


In [17]:
# yolo_model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # YOLOv5s 모델 로드

Using cache found in /home/nkw/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2024-8-11 Python-3.8.10 torch-2.4.0+cu121 CUDA:0 (NVIDIA GeForce RTX 3090, 24257MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [18]:
# 영상 데이터에서 mp pose landmark dot 데이터 추출 부분
raw_data = []

for fold in os.listdir(video_path):
    for video_name in os.listdir(os.path.join(video_path, fold)):
        video_name_parts = video_name.split('_')
        if len(video_name_parts) >= 3:  # video_name이 예상대로 3개 이상의 파트로 구성된 경우에만 처리
            try:
                if int(video_name_parts[-1].split('.')[0]) >= 30:  # video name에서 마지막 숫자 확인
                    label = 0 if video_name_parts[1] == 'normal' else 1
                    skel_data_n, skel_data_f = get_skeleton(os.path.join(video_path, fold, video_name), attention_dot, draw_line)
                    
                    # 디버깅 출력 추가
                    print(f"Processing {video_name} - Label: {label}")
                    
                    if skel_data_n:
                        seq_list_n = skel_data_n[:30]
                        seq_list_f = skel_data_f[:30]
                        raw_data.append({'key': label, 'value': seq_list_n})
                        raw_data.append({'key': label, 'value': seq_list_f})
                        print(f"Added data - Key: {label}")
            except ValueError:
                print(f"Error processing file: {video_name}")

random.shuffle(raw_data)
print(f"Final raw_data length: {len(raw_data)}")
print(f"Raw data contents: {raw_data}")

I0000 00:00:1723524907.414994 1328244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723524907.423709 1329344 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master

Processing 08_bag_30.mp4 - Label: 1
Added data - Key: 1


I0000 00:00:1723524913.162242 1328244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723524913.172951 1329382 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master

Processing 03_pocket_30.mp4 - Label: 1
Added data - Key: 1


I0000 00:00:1723524921.573580 1328244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723524921.582543 1329419 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master

...

Processing 09_normal_30.mp4 - Label: 0
Added data - Key: 0


I0000 00:00:1723525013.943207 1330053 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp

Processing 07_normal_30.mp4 - Label: 0
Added data - Key: 0


I0000 00:00:1723525019.637945 1328244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723525019.646901 1330090 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/home/nkw/.cache/torch/hub/ultralytics_yolov5_master

Processing 03_normal_30.mp4 - Label: 0
Added data - Key: 0
Final raw_data length: 40
Raw data contents: [{'key': 0, 'value': [[0.4363498091697693, 0.25077927112579346, 0.8052356541156769, 0.2357235997915268, 0.34291350841522217, 0.4300210773944855, 0.8488611876964569, 0.38547614216804504, 0.2918161153793335, 0.5870980620384216, 0.9260814487934113, 0.48366251587867737, 0.23058342933654785, 0.6265225410461426, 0.9391724765300751, 0.5160379409790039, 0.2704721689224243, 0.6298372149467468, 0.9335095882415771, 0.5095565319061279, 0.3031829595565796, 0.6170740127563477, 0.9092893004417419, 0.4996867775917053, 0.49744343757629395, 0.5176143050193787, 0.6734113395214081, 0.520902156829834, 0.4550364017486572, 0.7192281484603882, 0.5828977525234222, 0.7150613069534302, 0.30911028385162354, 0.851832926273346, 0.48581188917160034, 0.8751221895217896], [0.38892537355422974, 0.6974974274635315, 0.7910419404506683, 0.7316592335700989, 0.18693959712982178, 0.8196382522583008, 0.7777382135391235, 0.9

In [19]:
print(raw_data)  # raw_data 리스트의 내용을 출력

[{'key': 0, 'value': [[0.4363498091697693, 0.25077927112579346, 0.8052356541156769, 0.2357235997915268, 0.34291350841522217, 0.4300210773944855, 0.8488611876964569, 0.38547614216804504, 0.2918161153793335, 0.5870980620384216, 0.9260814487934113, 0.48366251587867737, 0.23058342933654785, 0.6265225410461426, 0.9391724765300751, 0.5160379409790039, 0.2704721689224243, 0.6298372149467468, 0.9335095882415771, 0.5095565319061279, 0.3031829595565796, 0.6170740127563477, 0.9092893004417419, 0.4996867775917053, 0.49744343757629395, 0.5176143050193787, 0.6734113395214081, 0.520902156829834, 0.4550364017486572, 0.7192281484603882, 0.5828977525234222, 0.7150613069534302, 0.30911028385162354, 0.851832926273346, 0.48581188917160034, 0.8751221895217896], [0.38892537355422974, 0.6974974274635315, 0.7910419404506683, 0.7316592335700989, 0.18693959712982178, 0.8196382522583008, 0.7777382135391235, 0.9189271926879883, 0.27339327335357666, 0.7970449924468994, 0.7441090643405914, 0.8295223712921143, 0.2777

In [20]:
# dataset 길이 출력

nd = 0
ad = 0
for i in range(len(raw_data)):
    if raw_data[i]['key'] == 0:
        nd += 1
    else:
        ad += 1
print('normal data:', nd, '| abnormal data:', ad)

normal data: 20 | abnormal data: 20


In [21]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

- 모델링

In [22]:
class MyDataset(Dataset):
    def __init__(self, seq_list):
        self.X = []
        self.y = []
        for dic in seq_list :
            self.y.append(dic['key'])
            self.X.append(dic['value'])

    def __getitem__(self, index):
        data = self.X[index]
        label = self.y[index]
        return torch.Tensor(np.array(data)), torch.tensor(np.array(int(label)))

    def __len__(self):
        return len(self.X)

In [23]:
split_ratio = [0.8, 0.1, 0.1]
train_len = int(len(raw_data) * split_ratio[0])
val_len = int(len(raw_data) * split_ratio[1])
test_len = len(raw_data) - train_len - val_len

print('{}, {}, {}'.format(train_len, val_len, test_len))

32, 4, 4


In [24]:
train_dataset = MyDataset(raw_data)
train_data, valid_data, test_data = random_split(train_dataset, [train_len, val_len, test_len])

train_loader = DataLoader(train_data, batch_size=BATCH_SIZE)
val_loader = DataLoader(valid_data, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [26]:
# LSTM
class skeleton_LSTM(nn.Module):
    def __init__(self):
        super(skeleton_LSTM, self).__init__()
        self.lstm1 = nn.LSTM(input_size=len(attention_dot) * 2, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm2 = nn.LSTM(input_size=128, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm3 = nn.LSTM(input_size=256, hidden_size=512, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout1 = nn.Dropout(0.1)
        self.lstm4 = nn.LSTM(input_size=512, hidden_size=256, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm5 = nn.LSTM(input_size=256, hidden_size=128, num_layers=NUM_LAYERS, batch_first=True)
        self.lstm6 = nn.LSTM(input_size=128, hidden_size=64, num_layers=NUM_LAYERS, batch_first=True)
        self.dropout2 = nn.Dropout(0.1)
        self.lstm7 = nn.LSTM(input_size=64, hidden_size=32, num_layers=NUM_LAYERS, batch_first=True)
        self.fc = nn.Linear(32,2)

    def forward(self, x) :
        x, _ = self.lstm1(x)
        x, _ = self.lstm2(x)
        x, _ = self.lstm3(x)
        x = self.dropout1(x)
        x, _ = self.lstm4(x)
        x, _ = self.lstm5(x)
        x, _ = self.lstm6(x)
        x = self.dropout2(x)
        x, _ = self.lstm7(x)
        x = self.fc(x[:,-1,:]) # x[배치 크기, 시퀀스 길이, 은닉 상태 크기], [:, -1, :] -> 마지막 시간 단계만 선택

        return x

In [27]:
# 모델 초기화

def init_model():
    global net, loss_fn, optim
    plt.rc('font', size = 10)
    net = skeleton_LSTM().to(device)
    loss_fn = nn.CrossEntropyLoss()
    optim = Adam(net.parameters(), lr=0.0001)

# epoch 카운터 초기화
def init_epoch():
    global epoch_cnt
    epoch_cnt = 0

# 모든 Log를 초기화
def init_log():
    global log_stack, iter_log, tloss_log, tacc_log, vloss_log, vacc_log, time_log
    plt.rc('font', size = 10)
    iter_log, tloss_log, tacc_log, vloss_log, vacc_log = [], [], [], [], []
    time_log, log_stack = [], []

In [28]:
def record_train_log(_tloss, _tacc, _time):
    # Train Log 기록
    time_log.append(_time)
    tloss_log.append(_tloss)
    tacc_log.append(_tacc)
    iter_log.append(epoch_cnt)

def record_valid_log(_vloss, _vacc):
    # Validation Log 기록
    vloss_log.append(_vloss)
    vacc_log.append(_vacc)

def last(log_list):
    # last 안의 마지막 숫자를 반환(print_log 함수에서 사용)
    if len(log_list) > 0:
        return log_list[len(log_list) - 1]
    else:
        return -1

def print_log():
    # 학습 추이 출력 : 소숫점 3자리까지
    train_loss = round(float(last(tloss_log)), 3)
    train_acc = round(float(last(tacc_log)), 3)
    val_loss = round(float(last(vloss_log)), 3)
    val_acc = round(float(last(vacc_log)), 3)
    time_spent = round(float(last(time_log)), 3)

    log_str = 'Epoch: {:3} | T_Loss {:5} | T_Acc {:5} | V_Loss {:5} | V_Acc {:5} | {:5}'.format(last(iter_log), train_loss, train_acc, val_loss, val_acc, time_spent)

    log_stack.append(log_str)
    
    # 학습 추이 그래프 출력
    hist_fig, loss_axis = plt.subplots(figsize=(10, 3), dpi=99)
    hist_fig.patch.set_facecolor('white')

    # Loss Line 구성
    loss_t_line = plt.plot(iter_log, tloss_log, label='Train_Loss', color='red', marker='o')
    loss_v_line = plt.plot(iter_log, vloss_log, label='Valid_Loss', color='blue', marker='s')
    loss_axis.set_xlabel('epoch')
    loss_axis.set_ylabel('loss')

    # Acc, Line 구성
    acc_axis = loss_axis.twinx()
    acc_t_line = acc_axis.plot(iter_log, tacc_log, label='Train_Acc', color='red', marker='+')
    acc_v_line = acc_axis.plot(iter_log, vacc_log, label='Valid_Acc', color='blue', marker='x')
    acc_axis.set_ylabel('accuracy')

    # 그래프 출력
    hist_lines = loss_t_line + loss_v_line + acc_t_line + acc_v_line
    loss_axis.legend(hist_lines, [l.get_label() for l in hist_lines])
    loss_axis.grid()
    plt.title('Learning history until epoch {}'.format(last(iter_log)))
    plt.draw()

    # 텍스트 로그 출력
    clear_output(wait=True)
    plt.show()
    for idx in reversed(range(len(log_stack))):
        print(log_stack[idx])

In [29]:
def clear_memory():
    if device != 'cpu':
        empty_cache()
    gc.collect()

# 학습 알고리즘
def epoch(data_loader, mode = 'train'):
    global epoch_cnt

    # 사용되는 변수 초기화
    iter_loss, iter_acc, last_grad_performed = [], [], False

    # 1 iteration 학습 알고리즘(for문을 나오면 1 epoch 완료)
    for _data, _label in data_loader:
        data, label = _data.to(device), _label.type(torch.LongTensor).to(device)

        # 1. Feed-forward
        if mode == 'train':
            net.train()
        else:
            # 학습때만 쓰이는 Dropout, Batch Mormalization을 미사용
            net.eval()

        result = net(data) # 1 Batch에 대한 결과가 모든 Class에 대한 확률값으로
        _, out = torch.max(result, 1) # result에서 최대 확률값을 기준으로 예측 class 도출( _ : 값 부분은 필요 없음, out : index 중 가장 큰 하나의 데이터)

        # 2. Loss 계산
        loss = loss_fn(result, label) # GT 와 Label 비교하여 Loss 산정
        iter_loss.append(loss.item()) # 학습 추이를 위하여 Loss를 기록

        # 3. 역전파 학습 후 Gradient Descent
        if mode == 'train':
            optim.zero_grad() # 미분을 통해 얻은 기울기를 초기화 for 다음 epoch
            loss.backward() # 역전파 학습
            optim.step() # Gradient Descent 수행
            last_grad_performed = True # for문을 나가면 epoch 카운터 += 1

        # 4. 정확도 계산
        acc_partial = (out == label).float().sum() # GT == Label 인 개수
        acc_partial = acc_partial / len(label) # ( TP / (TP + TM)) 해서 정확도 산출
        iter_acc.append(acc_partial.item()) # 학습 추이를 위하여 Acc. 기록

    # 역전파 학습 후 Epoch 카운터 += 1
    if last_grad_performed:
        epoch_cnt += 1

    clear_memory()

    # loss와 acc의 평균값 for 학습추이 그래프, 모든 GT와 Label 값 for 컨퓨전 매트릭스
    return np.average(iter_loss), np.average(iter_acc)

def epoch_not_finished():
    # 에폭이 끝남을 알림
    return epoch_cnt < maximum_epoch

- 학습 진행

In [30]:
# Training initialization
init_model()
init_epoch()
init_log()
maximum_epoch = EPOCH

# Training iteration

while epoch_not_finished():
    start_time = time.time()

    tloss, tacc = epoch(train_loader, mode = 'train')

    end_time = time.time()
    time_taken = end_time - start_time
    record_train_log(tloss, tacc, time_taken)

    with torch.no_grad():
        vloss, vacc = epoch(val_loader, mode = 'val')
        record_valid_log(vloss, vacc)

    print_log()

print('\n Training completed!')

Epoch: 700 | T_Loss 0.673 | T_Acc 0.583 | V_Loss 0.757 | V_Acc   0.5 | 1.067
Epoch: 699 | T_Loss 0.683 | T_Acc 0.583 | V_Loss 0.758 | V_Acc   0.5 | 1.086
Epoch: 698 | T_Loss 0.685 | T_Acc 0.583 | V_Loss  0.76 | V_Acc   0.5 | 1.051
Epoch: 697 | T_Loss  0.69 | T_Acc 0.583 | V_Loss 0.764 | V_Acc   0.5 | 1.069
Epoch: 696 | T_Loss 0.693 | T_Acc 0.583 | V_Loss 0.774 | V_Acc   0.5 | 1.048
Epoch: 695 | T_Loss 0.696 | T_Acc 0.583 | V_Loss 0.785 | V_Acc   0.5 | 1.044
Epoch: 694 | T_Loss 0.703 | T_Acc 0.583 | V_Loss 0.796 | V_Acc   0.5 | 1.064
Epoch: 693 | T_Loss 0.771 | T_Acc 0.528 | V_Loss 0.804 | V_Acc   0.5 | 1.083
Epoch: 692 | T_Loss 0.839 | T_Acc 0.556 | V_Loss 0.802 | V_Acc   0.5 | 1.046
Epoch: 691 | T_Loss 0.932 | T_Acc   0.5 | V_Loss  0.82 | V_Acc   0.5 | 1.066
Epoch: 690 | T_Loss 0.296 | T_Acc 0.889 | V_Loss  2.16 | V_Acc   0.0 | 1.042
Epoch: 689 | T_Loss  0.71 | T_Acc  0.75 | V_Loss 0.502 | V_Acc  0.75 | 1.041
Epoch: 688 | T_Loss 0.461 | T_Acc 0.778 | V_Loss  1.12 | V_Acc   0.5 | 1.055

In [31]:
# 정확도 검증
with torch.no_grad():
    test_loss, test_acc = epoch(test_loader, mode = 'test')
    test_acc = round(test_acc, 4)
    test_loss = round(test_loss, 4)
    print('Test Acc.: {}'.format(test_acc))
    print('Test Loss: {}'.format(test_loss))

Test Acc.: 0.25
Test Loss: 0.7965


- 1분 원본 영상으로 모델 테스트 하기

In [33]:
# 영상 resize 및 추출
# test_video_name = 'C_3_12_43_BU_SMC_10-14_12-17-14_CC_RGB_DF2_F2'
test_video_name = 'C_3_12_1_BU_DYA_07-31_16-15-01_CA_RGB_DF2_M1'
# test_video_path = f'/content/drive/MyDrive/Colab_Notebooks/Anomaly Detection/{test_video_name}.mp4'
test_video_path = f'/home/nkw/ihs/ihs2/zerobase_DL_project/Anomaly Detection/{test_video_name}.mp4'
cv2.destroyAllWindows()
cap = cv2.VideoCapture(test_video_path)
img_list = []

if cap.isOpened():

    while True:
        ret, img = cap.read()
        if ret:
            img = cv2.resize(img, (640, 640))
            img_list.append(img)
            # cv2_imshow(img)
            # cv2.waitKey(1)
        else:
            break

cap.release()
cv2.destroyAllWindows()

print('저장된 frame의 개수: {}'.format(len(img_list)))

저장된 frame의 개수: 180


In [34]:
"""Yolov5 + Mediapipe Version"""

net.eval()

length = 30 # frame 상태를 표시할 길이
out_img_list = []
dataset = []
status = 'None'
pose = mp_pose.Pose(static_image_mode=True, model_complexity=1, enable_segmentation=False, min_detection_confidence=n_CONFIDENCE)
print('시퀀스 데이터 분석 중...')

xy_list_list = []
for img in tqdm(img_list):
    res = yolo_model(img)
    res_refine = res.pandas().xyxy[0].values

    nms_human = len(res_refine)
    if nms_human > 0:
        for bbox in res_refine:
            xx1, yy1, xx2, yy2 = int(bbox[0])-10, int(bbox[1]), int(bbox[2])+10, int(bbox[3])
            if xx1 < 0:
                xx1 = 0
            elif xx2 > 639:
                xx2 = 639
            if yy1 < 0:
                yy1 = 0
            elif yy2 > 639:
                yy2 = 639

            start_point = (xx1, yy1)
            end_point = (xx2, yy2)
            if bbox[4] > y_CONFIDENCE:
                img = cv2.rectangle(img, start_point, end_point, (0, 0, 255), 2)

                c_img = img[yy1:yy2, xx1:xx2]
                results = pose.process(cv2.cvtColor(c_img, cv2.COLOR_BGR2RGB)) # Yolo 바운딩 box 안에서 landmark dot 추출
                if not results.pose_landmarks: continue
                xy_list = []
                idx = 0
                draw_line_dic = {}
                for x_and_y in results.pose_landmarks.landmark:
                    if idx in attention_dot:
                        xy_list.append(x_and_y.x)
                        xy_list.append(x_and_y.y)
                        x, y = int(x_and_y.x*(xx2-xx1)), int(x_and_y.y*(yy2-yy1))
                        draw_line_dic[idx] = [x, y]
                    idx += 1

                xy_list_list.append(xy_list)
                for line in draw_line:
                    x1, y1 = draw_line_dic[line[0]][0], draw_line_dic[line[0]][1]
                    x2, y2 = draw_line_dic[line[1]][0], draw_line_dic[line[1]][1]
                    c_img = cv2.line(c_img, (x1, y1), (x2, y2), (0, 255, 0), 3)

                if len(xy_list_list) == length:
                    dataset = []
                    dataset.append({'key' : 0, 'value' : xy_list_list})
                    dataset = MyDataset(dataset)
                    dataset = DataLoader(dataset)
                    xy_list_list = []

                    for data, label in dataset:
                        data = data.to(device)
                        with torch.no_grad():
                            result = net(data)
                            _, out = torch.max(result, 1)
                            if out.item() == 0: status = 'Normal'
                            else: status = 'Theft'

    cv2.putText(img, status, (0, 50), cv2.FONT_HERSHEY_COMPLEX, 1.5, (0, 0, 255), 2)
    out_img_list.append(img)

I0000 00:00:1723527563.321768 1328244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1723527563.338189 1331585 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 3090/PCIe/SSE2


시퀀스 데이터 분석 중...


  0%|          | 0/180 [00:00<?, ?it/s]/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  1%|          | 1/180 [00:00<00:54,  3.29it/s]/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  1%|          | 2/180 [00:00<00:47,  3.76it/s]/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
  2%|▏         | 3/180 [00:00<00:49,  3.56it/s]/home/nkw/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:869: FutureWarning: `torch.cuda.amp.autocast(args...)` is depre

In [35]:
# 테스트 원본 영상 내보내기
filename = './output.mp4'
fourcc = cv2.VideoWriter_fourcc(*'DIVX')
fps = 3
frameSize = (640, 640)
isColor = True
out = cv2.VideoWriter(filename, fourcc, fps, frameSize, isColor)
for out_img in out_img_list:
    out.write(out_img)
out.release()

OpenCV: FFMPEG: tag 0x58564944/'DIVX' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


- 모델 저장

In [37]:
# 모델 저장하기
PATH = '/content/drive/MyDrive/'
PATH = '/home/nkw/ihs/ihs2/zerobase_DL_project/'
model_name = 'LSTM.pt'
torch.save(net.state_dict(), PATH + model_name)